# How To Build a Pretty Good Wordle Bot

Author: Alex Spradling

---
A lot of folks have done this, and probably done it better, but this is how I did it. This notebook will cover building a WORDLE bot algorithm from the ground up. The final WORDLE bot solves all WORDLE problems in around 3.5 guesses on average which is very close to the "mathematically optimal" solution. I'm happy enough with the results and happier to share them with you. So here we go, here's how you build a WORDLE bot that is pretty good. 


Note to Readers: I know these Jupyter Notebooks seem intimidating. The combination of word blocks, codeblocks, and formulae are weird -- but I promise I'll spend  the majority of my time writing about the mathematical and logical intuitions behind its function. The code that carries it out is just a tool, so if the presence of the code blocks puts you off, fear not. This isn't a CS class. 

---

# WORDLE BOT! 

The heart of algorithm is based on INFORMATIONAL ENTROPY, which at its core is just some 9th grade math applied in a very clever way.

INFORMATIONAL ENTROPY was developed by Claude Shannon at Bell Labs in 1948. It provides a quantitative measure of the uncertainty or randomness in a set of outcomes. Let's delve into the foundational concepts and the mathematical formulation of entropy.

## Defining Key Concepts

1. **Information**: In the context of information theory, 'information' quantifies the reduction in uncertainty. When an event occurs that we were uncertain about, we gain information. The more uncertain the event, the more information it provides.

2. **Bit**: A 'bit' is the basic unit of information in information theory. It represents 1 unit of information in the form of a binary choice. A decision that can result in two equally likely outcomes provides one bit of information -- If I flip a fair coin and tell you I have Heads, you now know that the other side must be Tails -- we've eliminated 1 piece of uncertainty (and therefore gained 1 piece of information).

3. **Probability**: The chance of a random event happening. A basic probability is found by dividing your preferred event by all other possible events. (The probability of getting heads when flipping a fair coin is .5 -- one event/two possible outcomes = 1/2 = .5)


# 1. Building Inuition 
## Information as Uncertainty

If I flip a (fair) coin and tell you the side facing up is heads I've given you a little bit (to be precise, one bit) of information. 

You know, as a matter of fact, that a coin has two sides, heads and tails, and that it turns out that flipping a coin many times results in one side or the other coming up in my palm with equal probability. 

By telling you that Heads is up *you also know that tails is down* -- you've gained that 1 piece of information. 

Information Entropy rigorously reconciles this concept of uncertainty and information. 


## The Formula for Entropy

$$ H(X) = -\sum_{i=1}^{n} P(x_i) \log_2 P(x_i) $$

- **The Summation $\sum$**: Represents the sum of information from all possible outcomes.
- **Probability $P(x_i)$**: The likelihood of each outcome.
- **Logarithm $\log_2$**: A logarithm, in simple terms, tells us how many times we need to multiply a base number (like 2) by itself to get another number. For example, $\log_2 8 = 3$ because $2 \times 2 \times 2 = 8$. 
- **Why Base 2?**: In information theory, we often deal with scenarios that have two outcomes, like flipping a coin. Using a base-2 logarithm is natural here because it directly relates to these binary (two-option) scenarios. Each bit of information represents a choice between two equally likely possibilities. Thus, the base-2 logarithm measures the number of binary choices (or bits) needed to encode the information from an outcome.

For a fair coin (H for Heads, T for Tails), each with a probability of 0.5:

$$ H(X) = -[P(H) \log_2 P(H) + P(T) \log_2 P(T)] $$
$$ H(X) = -[0.5 \log_2 0.5 + 0.5 \log_2 0.5] $$
$$ H(X) = -[-0.5 - 0.5] $$
$$ H(X) = 1 \text{ bit} $$

So, we see that as defined above, 1 piece of information is an event with a binary outcome, I tell you I've got Heads, and since you know that we have an equal probability of getting Tails you can eliminate the uncertainty that the coin could be showing tails. That's the information you just gained. 

## Stay With Me!

Now, let's consider a six-sided die. Unlike the coin toss, a dice roll has more outcomes, each with its own probability. If I roll a fair dice and tell you that I rolled a 1, I've informed you of more than just the outcome. Implicitly, I've also told you that the dice *isn't showing 2, 3, 4, 5, or 6.*

This means you've gained more information from the result of a dice roll than from a coin flip because more potential outcomes are eliminated based on the lower probability of each outcome occurring. 

The Math:


Applying our entropy formula:

$$ H(X) = -\sum_{i=1}^{n} P(x_i) \log_2 P(x_i) $$

For a fair dice, our random variable $X$ can take six values (1, 2, 3, 4, 5, 6), each with a probability of 1/6. The calculation is:

$$ H(X) = -\sum_{i=1}^{6} \frac{1}{6} \log_2 \frac{1}{6} $$
$$ H(X) = -6 \times \left( \frac{1}{6} \log_2 \frac{1}{6} \right) $$
$$ H(X) = -\log_2 \frac{1}{6} $$
$$ H(X) \approx 2.58 \text{ bits} $$


Ok, so hopefully things are starting to make a little bit more sense now. We know a 1 on a fair die also means NOT 2,3,4,5,6, which is more information than just 1 is not 2. But as we look at the calculation above and see 2.58 bits as the result and the definition of a bit is "1 piece of information" which is further defined as a result that can be narrowed down to 2 equally likely outcomes we start to get confused again -- what exactly is 2.58 decisions that result in equally likely outcomes?



Consider you're trying to guess the outcome of a die roll:

In this scenario, the path to the answer may vary, sometimes being shorter or longer, which is why the average information content is a fractional number like 2.58. This represents the average amount of uncertainty resolved, or information gained, from observing the outcome of a fair die roll.
    
1. **First Question: "Is the number greater than 3?"**
   - If Yes: The possible outcomes are 4, 5, or 6.
   - If No: The possible outcomes are 1, 2, or 3.

2. **Second Question:**
   - If the answer to the first question was Yes:
     - Ask: "Is the number less than 5?"
       - If Yes: The outcome is 4.
       - If No: The outcomes are 5 or 6.
   - If the answer to the first question was No:
     - Ask: "Is the number greater than 1?"
       - If Yes: The outcomes are 2 or 3.
       - If No: The outcome is 1.

3. **Third Question:**
   - Only needed if there are still two possibilities after the second question.
     - If the remaining possibilities are 5 and 6:
       - Ask: "Is the number less than 6?"
         - If Yes: The outcome is 5.
         - If No: The outcome is 6.
     - If the remaining possibilities are 2 and 3:
       - Ask: "Is the number less than 3?"
         - If Yes: The outcome is 2.
         - If No: The outcome is 3.

In this scenario, the path to the answer may vary, sometimes being shorter or longer, which is why the average information content is a fractional number like 2.58. This represents the average amount of uncertainty resolved, or information gained, from observing the outcome of a fair die roll.

### More Certainty = Less Uncertainty to Eliminate = Less Information

Ok, you understand more uncertainty = more information. What does less uncertainty = less information look like?

Back to the coins. What if the coin was *not* fair, it's loaded and it will come up heads more often than tails. We start flipping the coin and it's coming up heads more than tails and this isn't really surprising to you because *you know* that I've skewed the probabilities and the fact that heads is coming up more than tails just isn't really news to you. So with less uncertainty, we gain a little less information. 

Let's do the math again, we'll say that heads comes up 70% of the time with my new fixed coin.

$$ H(X) = -[P_1 \log_2 P_1 + P_2 \log_2 P_2] $$
$$ H(X) = -[0.7 \log_2 0.7 + 0.3 \log_2 0.3] $$
$$ H(X) = -[0.7 \times -0.5146 + 0.3 \times -1.7370] $$
$$ H(X) = -[-0.3602 - 0.5211] $$
$$ H(X) \approx 0.8813 \text{ bits} $$

I've rigged the results and you are less surprised and therefore less informed by the results of the coin flip, we can see that the calculated information is now $.88$ bits where it was once $1$ when we had a fair coin. 

# 2. Practical Application

So now that we know that higher entropy = more uncertainty eliminated, and we know how to calculate entropy, we can turn to a practical example, WORDLE. 


Our goal here is to select words that eliminate the most uncertainty, and narrow down the population of possible answer choices as best we can. 

We'll start first with a list of words. WORDLE uses a curated list of 5 letter words that is 2316 words long, but for now it'll be a lot simpler to explain if we just use a much smaller list:

word_pool = `[
    "apple", "arise", "angle", "amble", "acorn", "alien", "align", "ankle", "agent", "adore",
    "baker", "blaze", "brine", "brace", "broil", "beach", "brain", "brief", "brave", "bread",
    "chair", "chase", "child", "choke", "climb", "clock", "cloud", "clown", "crown", "crisp",
    "dance", "drape", "drive", "drown", "drake", "dread", "dream", "dress", "drill", "drink",
    "eagle", "earth", "elbow", "elder", "elect", "email", "enter", "event", "every", "excel",
    "fairy", "faith", "false", "fancy", "feast", "field", "fiery", "fight", "final", "flame",
    "grace", "grade", "grain", "grand", "grant", "grape", "grass", "great", "green", "greet"
]`

We'll also need a secret_word = `"flame"`. This is the word we're trying to guess.

Ok, the game is set and the algo is on! 

Our goal is to algorithmically find the word that best divides the list using Entropy. 

Step 1 is to figure out a feedback system like the one uses in wordle. For our purposes we'll use this sytem: We'll traverse each letter of our guess and compare it to the secret word, if the letter is a match, it's a `+`, if the letter is in the `secret_word` but not in the correct position, we'll use a `-` and if the letter isn't in the `secret_word` we'll use a `0`. The function I'll code below called `get_feedback_pattern` does this work for us:


In [1]:
import sys
import os
import math
import random
import pandas as pd

# path to word_lists
word_lists_path = os.path.join(os.getcwd(), 'word_lists')

from word_lists.officialguesses import official_guesses
from word_lists.officialanswers import official_answers

In [2]:
def get_feedback_pattern(guess, answer):
    used_indices = set()  # Track used indices in the answer
    pattern = ['0'] * len(guess)  # Initialize feedback pattern with '0's

    # First pass: Check for correct letters in the correct position
    for i, (g, a) in enumerate(zip(guess, answer)):
        if g == a:
            pattern[i] = '+'
            used_indices.add(i)

    # Second pass: Check for correct letters in the wrong position
    for i, g in enumerate(guess):
        if pattern[i] == '0' and g in answer:
            # Ensure the letter is not already matched
            for j, a in enumerate(answer):
                if g == a and j not in used_indices:
                    pattern[i] = '-'
                    used_indices.add(j)
                    break

    return tuple(pattern)

And we'll use it below on an example guess, the word `great`

In [3]:
get_feedback_pattern(guess = 'great', answer = 'flame')

('0', '0', '-', '-', '0')

And above we see the coded feedback string `0+--0` for the guess `great` (remember we're trying to guess the word `flame`).

Next, we'll compare our guess to every list in our `word_pool` and get a dictionary of patterns and a count of their occurences in our `word_pool`

In [4]:
word_pool = [
    "apple", "arise", "angle", "amble", "acorn", "alien", "align", "ankle", "agent", "adore",
    "baker", "blaze", "brine", "brace", "broil", "beach", "brain", "brief", "brave", "bread",
    "chair", "chase", "child", "choke", "climb", "clock", "cloud", "clown", "crown", "crisp",
    "dance", "drape", "drive", "drown", "drake", "dread", "dream", "dress", "drill", "drink",
    "eagle", "earth", "elbow", "elder", "elect", "email", "enter", "event", "every", "excel",
    "fairy", "faith", "false", "fancy", "feast", "field", "fiery", "fight", "final", "flame",
    "grace", "grade", "grain", "grand", "grant", "grape", "grass", "great", "green", "greet"
]


guess = 'great'

pattern_counts = {}
for answer in word_pool:
    pattern = get_feedback_pattern(guess, answer)
    pattern_counts[pattern] = pattern_counts.get(pattern, 0) + 1

# get first 5 most common patterns

print('total_patterns:', len(pattern_counts))
print('Top 5 most common patterns by count:')
sorted(pattern_counts.items(), key=lambda x: x[1], reverse=True)[:5]

total_patterns: 31
Top 5 most common patterns by count:


[(('0', '0', '-', '-', '0'), 11),
 (('0', '+', '0', '0', '0'), 6),
 (('0', '+', '-', '-', '0'), 5),
 (('0', '0', '0', '0', '0'), 5),
 (('0', '-', '0', '-', '0'), 3)]

Above we have the first 5 rows of big table of patterns, the results show that as we compare the word `great` to our `word_pool` the pattern `('0', '0', '-', '-', '0')` occured 11 times, the pattern `('0', '+', '0', '0', '0')` occured 6 times and so on...In total the word `great` yielded 31 different feedback patterns against the `word_pool`. 

---
### Let's stop here and plant a quick flag. 

You might be tempted to think that a word that produces the most feedback patterns is the most effective word. This is not the case. 

- A word that generates the most feedback patterns may have a large number of unique patterns, but this doesn't necessarily mean it has the highest entropy.

- If most of these patterns are highly unlikely (i.e., they have low probability), and only a few patterns are significantly more likely, the overall entropy might not be maximized. This is because entropy considers both the number and the probability distribution of these patterns.

- For example, a guess that could equally likely be completely wrong, partially correct, or entirely correct (in terms of letter positions and presence) would have high entropy. In contrast, a guess that is almost always entirely wrong or right, with little variation, would have lower entropy despite possibly generating many feedback patterns.

---

### Ok, let's keep marching towards calculating the Entropy of `great`

We'll want to convert these 31 patterns into probabilities. Remember at the beginning we stated "A basic probability is found by dividing your preferred event by all other possible events." 

Taking 1 of the 31 patterns, `('0', '0', '-', '-', '0')` as our pattern, let's compute the probability of occurence. We know it occured 11 times in the `word_pool`, so the probability of this pattern occuring in the word pool is 11/ all patterns, or 70, giving us a probability of .1571. We do this for all 31 of the yielded patterns. Let's use the function defined below to do this:

In [5]:
total_patterns = len(word_pool)

def calculate_pattern_probabilities(pattern_counts, total_patterns):
    probabilities = {}
    for pattern, count in pattern_counts.items():
        probabilities[pattern] = count / total_patterns
    return probabilities

# get probabilities of 5 most common patterns

pattern_probabilities = calculate_pattern_probabilities(pattern_counts, total_patterns)


print('Top 5 most common patterns and their probabilities:')
sorted(pattern_probabilities.items(), key=lambda x: x[1], reverse=True)[:5]

Top 5 most common patterns and their probabilities:


[(('0', '0', '-', '-', '0'), 0.15714285714285714),
 (('0', '+', '0', '0', '0'), 0.08571428571428572),
 (('0', '+', '-', '-', '0'), 0.07142857142857142),
 (('0', '0', '0', '0', '0'), 0.07142857142857142),
 (('0', '-', '0', '-', '0'), 0.04285714285714286)]

Now we have a list of 31 probabilities for patterns, generated against the `word_pool`, using the word `great`. Phew. Almost there. 

Let's revisit the formula for entropy and get our bearings straight:

$$ H(X) = -\sum_{i=1}^{n} P(x_i) \log_2 P(x_i) $$

We've got all of our probabilities for our patterns, now its a matter of multiplying each probability by $\log_2 P(x_i)$ and summing everything up. The function below will do that for us:

In [6]:
def calculate_entropy(pattern_probabilities):
    entropy = 0
    for probability in pattern_probabilities.values():
        entropy -= probability * math.log(probability, 2)
    return entropy

In [7]:
entropy =  calculate_entropy(pattern_probabilities)
print('Entropy of the word "great":', entropy)

Entropy of the word "great": 4.510538320213657



So now the big question is, was `great` a great (ha) word to use? 

Is it effectively eliminating the most uncertainty? 

Another way of thinking of this is, is the word generating the most diverse set of feedback patterns? 

We'll need some results to know. Let's see how many of the original pool of 70 words are eliminated by the feedback patterns generated by `great`:


In [8]:
def refine_guess_list(guess_list, current_guess, feedback):
    refined_list = []

    for word in guess_list:
        remaining_letters = list(word)  # Track remaining letters
        match = True

        # First Pass: Correctly Placed Letters
        for i, letter in enumerate(current_guess):
            if feedback[i] == '+':
                if word[i] != letter:
                    match = False
                    break
                remaining_letters.remove(letter)  # Remove matched letter

        if not match:
            continue

        # Second Pass: Misplaced Letters
        for i, letter in enumerate(current_guess):
            if feedback[i] == '-':
                if letter not in remaining_letters or letter == word[i]:
                    match = False
                    break
                remaining_letters.remove(letter)  # Remove matched letter
            elif feedback[i] == '0' and letter in remaining_letters:
                match = False
                break

        if match:
            refined_list.append(word)

    return refined_list

In [9]:
print('Starting Pool of Words:', len(word_pool))

words_remaining = refine_guess_list(word_pool, 'great', get_feedback_pattern(guess = 'great', answer = 'flame'))

print('Words remaining after using "great" as a guess:', len(words_remaining))

# df of words remaining
words_remaining_df = pd.DataFrame(words_remaining, columns=['words remaining'])
words_remaining_df.head(11)

Starting Pool of Words: 70
Words remaining after using "great" as a guess: 11


,words remaining
0,apple
1,amble
2,alien
3,ankle
4,blaze
5,beach
6,chase
7,dance
8,email
9,false


So `great` seems like a pretty good word in this case. We've eliminated all but 11 of the 70 words in the pool and are well on our way to solving the `secret_word` (`flame`).


### But again we ask the question, was there a better word that eliminates the most uncertainty? 

We can find the answer by calculating the entropy of every word in our `word_pool` and seeing if there are higher entropy words -- words that that will eliminate even more uncertainty by generating the most evenly distributed feedback patterns. 

The code below this will calculate the entropy of every word in our `word_pool` and rank them:

In [10]:

def calculate_entropy(guess, answer_list):
    # Dictionary to hold counts of each feedback pattern
    pattern_counts = {}

    # Generate feedback patterns for each word in the answer list
    for answer in answer_list:
        pattern = get_feedback_pattern(guess, answer)
        pattern_counts[pattern] = pattern_counts.get(pattern, 0) + 1

    # Calculate entropy based on the frequency of each pattern
    entropy = 0
    total_patterns = len(answer_list)
    for count in pattern_counts.values():
        probability = count / total_patterns
        entropy -= probability * math.log(probability, 2)

    return entropy

# data frame of words and their entropy
df = pd.DataFrame(word_pool, columns=["word"])

# add entropy column
df["entropy"] = df["word"].apply(lambda x: calculate_entropy(x, word_pool))

df.sort_values(by="entropy", ascending=False, inplace=True)

df.head(10)

,word,entropy
13,brace,5.166653
12,brine,5.086516
60,grace,5.068359
61,grade,5.060797
30,dance,5.003984
1,arise,4.998408
63,grand,4.975083
16,brain,4.968449
34,drake,4.955869
31,drape,4.927298


So it looks like `great` isn't even in the top 10, and `brace` is the best word we could have picked!

Well, the proof is always in the pudding, so how many words remain if we use `brace` instead of `great` as our guess. The code below this will filter the list:

In [11]:
refine_guess_list(word_pool, 'brace', get_feedback_pattern('brace', 'flame'))

['flame']



Well how about that! `brace` filters the list down to just one word -- Looks like Bell Labs didn't employ many slouches. 

Of course this example used 70 words and the current version of WORDLE uses 2315 official answers and 14854 acceptable guesses. So the computational challenge scales up a little but, but not much --  I combined the functions found above in this notebook and created ALEX BOT, a pretty good wordle bot that isn't perfectly optimal but close to it and is fun to play around with, you can find it in this directory. 

--- 

# So what's the best word to start a game of WORDLE with? 

In [12]:
guess_pool = official_guesses
answer_pool = official_answers

First let's find the top 20 5-letter words ranked by Entropy

In [13]:
# data frame of entropy for every word in the guess_pool against the answer_pool
entropy_df = pd.DataFrame(guess_pool, columns=["word"])
entropy_df["entropy"] = entropy_df["word"].apply(lambda x: calculate_entropy(x, answer_pool))
entropy_df.sort_values(by="entropy", ascending=False, inplace=True)
entropy_df.head(10)


,word,entropy
12822,TARSE,5.946725
13077,TIARE,5.931203
12015,SOARE,5.885960
10767,ROATE,5.882779
10286,RAISE,5.877910
10280,RAILE,5.865710
10435,REAST,5.865457
11829,SLATE,5.855775
2637,CRATE,5.834874
11095,SALET,5.834582


In [14]:
# make a list of the top 10 words with the highest entropy
top_20_words = entropy_df["word"].tolist()[:20]
top_20_words

['TARSE',
 'TIARE',
 'SOARE',
 'ROATE',
 'RAISE',
 'RAILE',
 'REAST',
 'SLATE',
 'CRATE',
 'SALET',
 'IRATE',
 'TRACE',
 'SATER',
 'ARISE',
 'ORATE',
 'STARE',
 'CARTE',
 'RAINE',
 'RANSE',
 'CARET']

Now we'll use `AlexBot` to guess each of the 2315 words using each of the top 20 words as the starting guess. We'll take the mean of the number of guesses across all 2315 words and see which word has the lowest mean. 

In [15]:
%%capture
from alex_bot import simulate_all_wordle_games, load_word_list_from_py
GUESS_LIST_PATH = "word_lists/officialanswers.py"
ANSWER_LIST_PATH = "word_lists/officialanswers.py"
guess_list = load_word_list_from_py(GUESS_LIST_PATH)
answer_list = load_word_list_from_py(ANSWER_LIST_PATH)

# simulate all wordle games for the top 10 words, make a data frame of the results
results = pd.DataFrame()
results["word"] = top_20_words
results['average_guesses'] = results["word"].apply(lambda x: simulate_all_wordle_games(x, guess_list, answer_list))
results.sort_values(by="average_guesses", ascending=True, inplace=True)
results.head(10)

In [16]:
results.head(10)

,word,average_guesses
9,SALET,3.571058
0,TARSE,3.580994
7,SLATE,3.581425
11,TRACE,3.583585
6,REAST,3.586609
16,CARTE,3.599136
8,CRATE,3.602160
19,CARET,3.613823
18,RANSE,3.615551
15,STARE,3.619006


# So, `SALET` is the winner(sorta)

What's a `SALET`? The internet tells me it's some sort of helmet. The things you learn. 

The truth of the matter is that as of the date of the publication of this little notebook, there might be a better word, the NY Times now controls WORDLE and has been changing the word dataset. I believe that my `word_pools` are a mixture of pre and post nytimes WORDLE datasets and `TARSE` (going by the entropy numbers) is probably the best starting word, but it's impossible to validate without the current dataset, which is no longer publicly available. 

Still, there's only so many 5-letter words, so `SALET` is likely in the running for top words. So, put your SALET on and WORDLE on!